# Введение в графовые алгоритмы

## Представление графов

## Базовые алгоритмы

### Обход в глубину

**Идея:**

In [1]:
#include <vector>

using namespace std;

const uint32_t max_n = 1e5;
bool visited[max_n];
vector<vector<uint32_t>> graph;

void dfs(const uint32_t start) {
    visited[start] = true;

    for (const auto v : graph[start]) {
        if (!visited[v]) {
            dfs(v);
        }
    }
}

Асимптотика $O(V + E)$ time и $O(V)$ памяти

Немного его модифицируем, а именно будем сохранять для каждой вершины, в какой момент мы в неё вошли и в какой вышли — соответствующие массивы будем называть t_in и t_out.

In [1]:
#include <vector>

using namespace std;

const uint32_t max_n = 1e5;
bool visited[max_n];
uint32_t t_in[max_n];
uint32_t t_out[max_n];
vector<vector<uint32_t>> graph;
uint32_t t = 0;

void dfs(const uint32_t start) {
    t_in[start] = t;
    t++;

    for (const auto v : graph[start]) {
        if (!visited[v]) {
            dfs(v);
        }
    }

    t_out[start] = t;
}

У этих массивов много полезных свойств:

Вершина 
u
u является предком 
v
v 
⟺
t
i
n
v
∈
[
t
i
n
u
,
t
o
u
t
u
)
⟺tin 
v
​
 ∈[tin 
u
​
 ,tout 
u
​
 ). Эту проверку можно делать за константу.
Два полуинтервала 
[
t
i
n
v
,
t
o
u
t
v
)
[tin 
v
​
 ,tout 
v
​
 ) и 
[
t
i
n
u
,
t
o
u
t
u
)
[tin 
u
​
 ,tout 
u
​
 ) либо не пересекаются, либо один вложен в другой.
В массиве 
t
i
n
tin есть все числа от 0 до 
(
n
−
1
)
(n−1), причём у каждой вершины свой номер.
Размер поддерева вершины 
v
v (включая саму вершину) равен 
(
t
o
u
t
v
−
t
i
n
v
)
(tout 
v
​
 −tin 
v
​
 ).
Если ввести нумерацию вершин, соответствующую 
t
i
n
tin-ам, то индексы любого поддерева всегда будут каким-то промежутком в этой нумерации.

## Поиск компонент связности

In [ ]:
#include <vector>

using namespace std;

const uint32_t max_n = 1e5;
bool visited[max_n];
uint32_t components[max_n];
vector<vector<uint32_t>> graph;

void dfs(const uint32_t start, const uint32_t component_number) {
    visited[start] = true;
    components[start] = component_number;

    for (const auto v : graph[start]) {
        if (!visited[v]) {
            dfs(v, component_number);
        }
    }
}

int32_t components_amount(uint32_t n) {
    int amount = 0;
    for (uint32_t i = 0; i < n; i++) {
        if (!visited[i]) {
            dfs(i, amount);
            amount++;
        }
    }

    return amount;
}

## Проверка графа на двудольность

In [1]:
#include <iostream>
#include <vector>

using namespace std;

const uint32_t max_n = 1e5;
int colors[max_n];
vector<vector<uint32_t>> graph;

void dfs(const uint32_t start, const uint32_t color) {
    colors[start] = 1;

    for (const auto v : graph[start]) {
        if (!colors[v]) {
            dfs(v, -color);
        } else if (colors[v] != -color) {
            cout << "Graph is not bipartite" << endl;
            exit(0);
        }
    }
}

void is_bipartite(uint32_t n) {
    for (int v = 0; v < n; v++)
        if (!colors[v]) {
            dfs(v, 1);
        }
}

Однако про раскраски можно доказать некоторые полезные утверждения, например:

Граф можно раскрасить в два цвета — или, что эквивалентно, граф является двудольным — тогда и только тогда, когда все его простые циклы имеют чётную длину.
Если степень любой вершины не превосходит 
k
k, то граф можно раскрасить в 
k
k цветов.
Любое планарный граф возможно раскрасить в 4 цвета.

## Нахождение цикла

In [ ]:
#include <vector>
#include <iostream>

using namespace std;

const uint32_t max_n = 1e5;
bool visited[max_n];
vector<vector<uint32_t>> graph;

void dfs(const uint32_t start, const uint32_t p = -1) {
    if (visited[start]) {
        cout << "Graph has a cycle" << endl;
        exit(0);
    }

    visited[start] = true;

    for (const auto v : graph[start]) {
        if (v != p) {
            dfs(v, start);
        }
    }
}

void has_cycle(uint32_t n) {
    for (int v = 0; v < n; v++) {
        if (!visited[v]) {
            dfs(v);
        }
    }
}

## Топологическая сортировка

Мы хотим пронумеровать вершины орграфа так, чтобы все рёбра вели из вершины с меньшим номеров в вершину с большим. Очевидно, если в графе есть цикл, то этого сделать не получится, поэтому будем рассматривать только

**О.** Ацкиклический орграф (DAG) $G = \langle V, E \rangle$ - орграф : $\nexists v \in V \ (v, v) \in E^*$, т.е. это в точности граф без циклов

Очевидно, что для DAG верно: $\langle V, \prec \rangle$ - poset, где $\prec \ = \ E^*$ - отношение достижимости.

Более того, любой poset можно задать с помощью соответствующего DAG, если рассматривать в том числе бесконечные орграфы, но мы этого делать не будем

**О.** Задача топологической сортировки DAG (Topologicl sorting)

Гиперпараметры: $\langle X, <\rangle$ - линейно упорядоченное множество

Вход: $G = \langle V, E \rangle$ - DAG

Выход: $\tau: \langle V, \prec \rangle \longrightarrow \langle X, <\rangle$ - инъективная функция, сохраняющая порядок

То есть топологическая сортировка - это продолжение частичного порядка в какой-то заранее заданный линейный

**Идея:** Вершину, из которой не идет ни одно ребро можно поставить последней и выкинуть из графа. Затем можно итеративно повторять это размышление, пока не закончатся вершины

Рассматривать будем топологическую сортировку в $\langle \mathbb{N}, < \rangle$.

Алгоритм реализующий (и слегка улучшающий) эту идею (топологическая сортировка Тарьяна):

In [ ]:
#include <vector>
#include <iostream>

using namespace std;

vector<bool> visited;
vector<vector<uint32_t>> graph;
vector<uint32_t> t;

void dfs(const uint32_t start) {
    visited[start] = true;

    for (const auto v : graph[start]) {
        if (!visited[v]) {
            dfs(v);
        }
    }

    t.push_back(start);
}

void topological_sort() {
    for (uint32_t i = 0; i < graph.size(); i++) {
        if (!visited[i]) {
            dfs(i);
        }
    }
    reverse(t.begin(), t.end());
}


graph = {{1, 2}, {2, 3}, {3, 4}, {}, {}};
visited = vector<bool>(graph.size(), false);
topological_sort();
for (uint32_t k = 0; k < 5; k++) {
    cout << t[k] << " ";
}

**Теорема:**

$|V| \le |X| \Longrightarrow \exists \tau$ - топологическая сортировка

$\square$ Можно просто обобщить алгоритм предложенный выше $\blacksquare$

Асимптотика этого алгоритма очевидно $O(V + E)$

У DAG может быть несколько топологических сортировок, мы зачастую хотим выбрать какую-то конкретную из них, обладающую дополнительными свойствами. Для решения этой проблемы придумаем новый алгоритм

**Идея:** Итерируем граф, поддерживая очередь всех вершин с нулевой входной степенью. Каждый раз ставим такую вершину в начало, удаляем из графа, уменьшаем входные степени у её соседей

Теперь на этапе доставания вершин с нулевой степенью из очереди у нас есть контроль. Например, можем заменить queue на priority_queue по номеру вершины и тогда получим лексикографически минимальную топологическую сортировку

Алгоритм реализующий эту идею (топологическая сортировка Кана):

In [ ]:
#include <vector>
#include <queue>
#include <iostream>

using namespace std;

vector<uint32_t> topological_sort(const vector<vector<int>>& graph) {
    vector<uint32_t> in_deg(graph.size(), 0);
    for (auto & u : graph) {
        for (const uint32_t v : u) {
            ++in_deg[v];
        }
    }

    priority_queue<uint32_t, vector<uint32_t>, greater<>> q;
    for (uint32_t v = 0; v < graph.size(); ++v) {
        if (in_deg[v] == 0) {
            q.push(v);
        }
    }

    vector<uint32_t> order;
    order.reserve(graph.size());

    while (!q.empty()) {
        uint32_t u = q.top();
        q.pop();
        order.push_back(u);

        for (uint32_t v : graph[u]) {
            if (--in_deg[v] == 0) {
                q.push(v);
            }
        }
    }

    if (order.size() != graph.size()) {
        return {}; // есть цикл
    }

    return order;
}


vector<vector<int>> graph = {{1, 2}, {2, 3}, {3, 4}, {}, {}};
const auto order = topological_sort(graph);

for (uint32_t k = 0; k < graph.size(); k++) {
    cout << order[k] << " ";
}

Так же у нас теперь есть довольно простой способ проверять граф на циклы, что удобно, если мы не уверены, что на входе у нас будет точно DAG

**Теорема:**

Алгоритм Тарьяна построил топологическую сортировку $\Longleftrightarrow$ Алгоритм Кана построил топологическую сортировку

$\square$

Алгоритм Тарьяна расставляет из конца в начало, а Алгоритм Кана из начала в конец, но идея у них в общем-то похожая.

Строго доказывать не буду
$\blacksquare$

## Компоненты сильной связности

Пусть $G = \langle V, E \rangle$ - орграф

**О.** $C \subset V$ называется компонентой сильной связности (SCC), если $\forall v, u \in C \ \Longrightarrow \ v E^* u$ и $u E^* v$ и $C$ максимально по включению

Очевидные свойства SCC:

(1) $\forall v \in V \ \ \exists C : v \in C$

(2) $\forall C, C' \ \Longrightarrow \ C = C'$ или $C \cap C' = \emptyset$

т.о. $V = \bigsqcup C_i$, где $C_i$ - SCC

Придумаем алгоритм для нахождения такого разбиения

**Идея:** Запускаем DFS из любой вершины. Каждую вершину при входе в нее будем сохранять в стек. Так же будем поддерживать два параметра для каждой вершины: in_times - время входа в вершину, low_in_times - минимальное значение по:
1. index всех дочерних вершин по дереву обхода DFS (если пришли в новую вершину)
2. low дочерней вершины по дереву обхода DFS (если пришли в старую вершину, т.е. замкнули цикл)

Теперь, при возврате из рекурсии, если у вершины in_times == low_in_times, то мы точно знаем, что это корень SCC. В таком случае мы начинаем доставать из стека вершины, пока не дойдем то текущий. Это и будет компонента сильной связности

In [ ]:
#include <vector>
#include <stack>
#include <iostream>

using namespace std;

vector<vector<uint32_t>> graph;
uint32_t timer;
vector<uint32_t> in_times;
vector<uint32_t> low_in_times;
stack<uint32_t> backlink_stack;
vector<bool> on_backlink_stack;
vector<vector<uint32_t>> sccs;

void find_sscs_in_one_component(uint32_t v) {
    in_times[v] = low_in_times[v] = timer++;
    backlink_stack.push(v);
    on_backlink_stack[v] = true;

    for (const uint32_t to : graph[v]) {
        if (in_times[to] == -1) {
            find_sscs_in_one_component(to);
            low_in_times[v] = min(low_in_times[v], low_in_times[to]);
        } else if (on_backlink_stack[to]) {
            low_in_times[v] = min(low_in_times[v], in_times[to]);
        }
    }

    if (in_times[v] == low_in_times[v]) {
        vector<uint32_t> comp;
        while (true) {
            uint32_t u = backlink_stack.top();
            backlink_stack.pop();
            on_backlink_stack[u] = false;
            comp.push_back(u);
            if (u == v) break;
        }
        sccs.push_back(move(comp));
    }
}

void find_sccs() {
    sccs.clear();
    timer = 0;
    in_times = vector<uint32_t>(graph.size(), -1);
    low_in_times = vector<uint32_t>(graph.size(), 0);
    on_backlink_stack = vector<bool>(graph.size(), false);
    backlink_stack = stack<uint32_t>();

    for (uint32_t v = 0; v < graph.size(); ++v) {
        if (in_times[v] == -1) {
            find_sscs_in_one_component(v);
        }
    }
}


graph = {
    {1, 5}, {3, 5}, {3, 4, 7, 8}, {0, 6}, {7, 8}, {0, 3, 6}, {0}, {2, 4}, {7}
}; // https://upload.wikimedia.org/wikipedia/commons/6/6e/Algorithm_Tarjan.png

find_sccs();

cout << "Number of SCCs: " << sccs.size() << "\n";
for (size_t i = 0; i < sccs.size(); ++i) {
    cout << "SCC #" << i << ":";
    for (const uint32_t v : sccs[i]) {
        cout << ' ' << v;
    }
    cout << '\n';
}

Очевидно, асимптотика этого алгоритма $O(V + E)$

**Теорема**

Алгоритм Тарьяна корректно находит разбиение графа G на SCC

Рассмотрим связанную с SCC полезную конструкцию:

**О.** Конденсация графа $\mathcal{Cond}(G) = \langle V', E' \rangle$, где
1. $V' = \{C_i : C_i - \text{SCC графа} \ G\}$
2. $E' = \{(C_i, C_j) \ : \ \exists u \in C_i, v \in C_j \ : \ (u, v) \in E\}$

Понятно, что $\mathcal{Cond}(G)$ - DAG

На основании посчитанных SCC легко можно построить конденсацию:

In [ ]:
#include <unordered_map>

vector<vector<uint32_t>> condensation;

void build_condensation() {
    find_sccs();

    for (size_t i = 0; i < sccs.size(); ++i) {
        for (const uint32_t v : sccs[i]) {
            scc_map[v] = i;
        }
    }

    condensation = vector<vector<uint32_t>>(sccs.size());
    for (uint32_t v = 0; v < graph.size(); ++v) {
        for (const uint32_t to : graph[v]) {
            const uint32_t scc_v = scc_map[v];
            uint32_t scc_to = scc_map[to];
            if (scc_v != scc_to) {
                condensation[scc_v].push_back(scc_to);
            }
        }
    }
}


graph = {
    {1, 5}, {3, 5}, {3, 4, 7, 8}, {0, 6}, {7, 8}, {0, 3, 6}, {0}, {2, 4}, {7}
}; // https://upload.wikimedia.org/wikipedia/commons/6/6e/Algorithm_Tarjan.png

build_condensation();

for (size_t i = 0; i < condensation.size(); ++i) {
    cout << i << ":";
    for (const uint32_t v : condensation[i]) {
        cout << ' ' << v;
    }
    cout << '\n';
}

Асимптотика этого алгоритма $O(E)$